# SCAR
Recombination-aware phylogeographic inference using the structured coalescent with ancestral recombination.

SCAR Tutorial

by Fangfang Guo, David Rasmussen

This tutorial demonstrates the basics of setting up and running a model in SCAR. Here we will estimate migration rate and recombination rate of a simulated ARG (Ancestral Recombination Graph) in tree sequence format. The samples are supposed to sampling from 2 subpopulations. The full script can be found in the test_rho_mig_estimates.py under the route ./Simulations/EstimatesPrameters/Jointly_Est_M_rho. Other examples to estimate one single parameter can also be found in the folder EstimatesPrameters.     

Setting up SCAR

First we will import some standard Python packages in the codes SCARLikelihood and MCMC, which are the core codes of SCAR. In the code ARGSimulator, we use msprime to simulate ARGs. These packages can be installed via conda or pip if you do not already have them. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.linalg import expm
import msprime

After you have these packages and start to run the wrraper script test_rho_mig_estimates.py, it will import following packages:

In [ ]:
import MCMC
import ARGSimulator
import numpy as np

Setting up the model parameters

Now we need to parameterize the model of structured coalescent with ancestral recombination. There are five required parameters that need to be initialized.
1. Ne: effective population size.
2. genome_length: length of study genome length.
3. rho: the recombination rate of the genome length per generation
4. sample_sizes: the sample size in each subpopulations.
5. M: migration rate matrix.

Then we will set the parameters in the simulation. Here we conduct 100 simulations, with 200 samples and 2 subpopulation with symmetric migration rate.

In [ ]:
sims = 100 # simulations
Ne = 1.0  # effective pop sizes
genome_length = 1e4
M = [[0.0,mig_rate], [mig_rate,0.0]]  # migration rate matrix
samples = 100
sample_sizes = [samples]*2 # should be 1xpops vector
sim_rho_vals = np.linspace(0.1,10.0,num=sims)
sim_mig_vals = np.linspace(0.1,2.0,num=sims)

And in the simulation s, we set the true value of migration rate and recombination rate as

In [ ]:
mig_rate = sim_mig_vals[s] # the symmetric migration rate between 2 subpopulations
rho = sim_rho_vals[s] # the recombination rate on the genome_length region

These parameters are strored into a params dictionary,

In [ ]:
params = {'Ne': Ne, 'rho': rho, 'M': M, 'genome_length': genome_length, 'sample_sizes':  sample_sizes}

And what parameters you want to estimate also put into a dictionary, where 'True' is indicating the estimated parameter, while 'False' not. Here we will estimate 'rho' and 'M'. 

In [ ]:
est_params = {'Ne': False, 'rho': True, 'M': True}

We also need set the rows and columns of the 'true_fit_vals' and 'est_fit_vals', which store the true value of the estimated parameters and estimated value of parameters, respectively. In the 'est_fit_vals', there are 2.5% confidential interval, median value, and 97.5% confidential interval for each parameter. So we set both arrays as   

In [ ]:
true_fit_vals = np.zeros([sims,2])
est_fit_vals = np.zeros([sims,6])

Setting up time interval

SCAR approximates the lineages states by tracking how the probability of a lineage residing in each state changes backwards through time and integrate over their unknown states by small time interval dt_step. We need to change this parameter based on the maximum root time, and the smaller the time interval the more accurate approimate. Here in the SCARLikelihood, we set the time interval as 0.01 and unknown the ancestral states.    

In [ ]:
dt_step = 0.01
known_ancestral_states = False

Setting up the Bayesian MCMC parameters

There are four parameters you may need to change:
1. prior: the prior distribution on the demographic parameters.
2. optimize iteration: optimize proposal covariance matrix based on the samples in this iteration.
3. iterations: the total iteration you want to run.
4. burn in: how many iterations you want to discard.
Here we set a uniform prior for all the parameters, and iteration related parameters are setting as following: 

In [ ]:
prior = 1.0 # assume uniform prior
opt_iters = [200,500,1000,2000,5000] # the optimize iterations
estimates = MCMC.sample(params,est_params,ts,iterations=10000,opt_iters=opt_iters,burnin=1000,plot=True,**plot_info)

Saving the estimates

Finally, we will save the estimated recombination rate and migration rate by putting them into an array and then writing them to a csv file:

In [ ]:
estimates = MCMC.sample(params,est_params,ts,iterations=10000,opt_iters=opt_iters,burnin=1000,plot=True,**plot_info)
est_fit_vals[s,:] = estimates[:,0]
np.savetxt('test_SCAR_mcmc_rho_mig_unknownStates_ests.csv', est_fit_vals, delimiter=',')

SCAR is maintained by [davidrasm](https://github.com/davidrasm/) and [sunnyfangfangguo](https://github.com/sunnyfangfangguo/).